From the main directory "Ontolearn", run the commands for NCES data mentioned [here](https://ontolearn-docs-dice-group.netlify.app/usage/02_installation#download-external-files) to download pretrained models and datasets.

## Inference with NCES

In [1]:
from ontolearn.concept_learner import NCES
from ontolearn.knowledge_base import KnowledgeBase
from ontolearn.metrics import F1
from owlapy.parser import DLSyntaxParser

/home/nkouagou/.conda/envs/onto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", quality_func=F1(), num_predictions=100, learner_name="SetTransformer",
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", load_pretrained=True, max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32, pretrained_model_name="SetTransformer")

/home/nkouagou/.conda/envs/onto/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
KB = KnowledgeBase(path=nces.knowledge_base_path)

In [4]:
dl_parser = DLSyntaxParser(nces.kb_namespace)

### Let's learn the concept ``Father''

In [5]:
father = dl_parser.parse('Father')

In [6]:
not_father = dl_parser.parse('¬Father') # For negative examples

In [7]:
pos = set([ind.str.split("/")[-1] for ind in KB.individuals(father)])

In [8]:
neg = set([ind.str.split("/")[-1] for ind in KB.individuals(not_father)])

#### Prediction with SetTransformer

In [9]:
nodes = nces.fit(pos, neg).best_hypotheses(3)

In [10]:
nodes

[<class 'ontolearn.search.NCESNode'> at 0x7485924	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0xb329d38	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7c09448	Grandfather ⊔ Father	Quality:1.0	Length:3	|Indv.|:60]

In [11]:
nces.sorted_examples

False

#### Prediction with GRU

In [12]:
nces.pretrained_model_name = 'GRU'
nces.refresh()
nodes = list(nces.fit(pos, neg).best_predictions)

In [13]:
nodes[:5]

[<class 'ontolearn.search.NCESNode'> at 0x7485954	Father ⊔ Grandfather	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7c09948	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7c27918	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7c27944	Father ⊔ Father ⊔ Grandfather	Quality:1.0	Length:5	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7486610	Father	Quality:1.0	Length:1	|Indv.|:60]

#### Prediction with LSTM

In [14]:
nces.pretrained_model_name = 'LSTM'
nces.refresh()
nodes = list(nces.fit(pos, neg).best_predictions)

In [15]:
nodes[:5]

[<class 'ontolearn.search.NCESNode'> at 0x74859e0	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7c096b4	Father ⊔ Father ⊔ Grandfather	Quality:1.0	Length:5	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x74685d4	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7c22228	Father ⊔ (Person ⊓ (Grandfather ⊔ (∃ hasParent.(¬Parent))))	Quality:1.0	Length:10	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7487460	Father	Quality:1.0	Length:1	|Indv.|:60]

#### Prediction with ensemble SetTransformer+GRU+LSTM

In [16]:
nces.pretrained_model_name = ['SetTransformer','GRU','LSTM']
nces.refresh()
nodes = list(nces.fit(pos, neg).best_predictions)

In [17]:
nodes[:5]

[<class 'ontolearn.search.NCESNode'> at 0x7485980	Father ⊔ Father	Quality:1.0	Length:3	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7486500	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7472594	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7486460	Father	Quality:1.0	Length:1	|Indv.|:60,
 <class 'ontolearn.search.NCESNode'> at 0x7486490	Father ⊔ Grandfather	Quality:1.0	Length:3	|Indv.|:60]

### Scalability of NCES (solving multiple learning problems in a go!)

In [18]:
import json, time

In [19]:
with open("../NCESData/family/training_data/Data.json") as file:
    data = list(json.load(file).items())
## The function below takes an iterable of tuples
LPs = list(map(lambda x: (x[0], x[1]["positive examples"], x[1]["negative examples"]), data))

In [20]:
## We solve 256 learning problems!
lps = LPs[:256]

In [21]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False) ## Now predict with nces.fit_from_iterable.
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  2.310948610305786 seconds!


In [22]:
print(concepts[0])

OWLClass(IRI('http://www.benchmark.org/family#','Sister'))


In [23]:
print(len(concepts))

256


### Change pretrained model name, e.g., use ensemble model prediction

In [24]:
nces.pretrained_model_name = ['SetTransformer', 'GRU']

In [25]:
nces.pretrained_model_name

['SetTransformer', 'GRU']

In [26]:
nces.refresh()

In [27]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False)
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  3.00120210647583 seconds!


In [28]:
concepts[-1]

OWLObjectAllValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','married')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','PersonWithASibling'))))

## Training NCES

In [29]:
import json, time
with open("../NCESData/family/training_data/Data.json") as file:
    data = json.load(file) # Training data. Below we use the first 200 datapoints to train the synthesizer. You train on the full data, e.g. on GPU by remobing "[:200]"

In [30]:
from ontolearn.concept_learner import NCES

In [31]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", learner_name="SetTransformer",
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32,
            load_pretrained=False, pretrained_model_name="SetTransformer")

In [32]:
nces.train(list(data.items())[-200:], epochs=20, learning_rate=0.001, save_model=True, storage_path=f"./NCES-{time.time()}/")

********************Trainable model size********************
Synthesizer:  515296
********************Trainable model size********************

Training on CPU, it may take long...

##################################################

SetTransformer starts training... 

################################################## 



Loss: 0.5600, Soft Acc: 73.91%, Hard Acc: 70.67%: 100%|██████████| 20/20 [00:33<00:00,  1.66s/it]

Top performance: loss: 0.5600, soft accuracy: 74.71% ... hard accuracy: 72.45%

SetTransformer saved
